In [ ]:
import json
import os
from typing import Dict, List, Optional
from dataclasses import dataclass, field
from collections import defaultdict

teams = [
    "Complexity", "OG", "HEROIC", "Chinggis Warriors", "B8", "Imperial",
    "BetBoom", "Nemiga", "TYLOO", "NRG", "Lynn Vision", "Legacy",
    "Wildcard", "Metizport", "FlyQuest", "Fluxo", "Falcons", "FAZE",
    "3DMAX", "Virtus.pro", "FURIA", "MIBR", "paiN", "M80",
    "Aurora", "G2", "Natus Vincere", "Liquid", "Vitality", "MOUZ",
    "Spirit", "The MongolZ"
]

stages = ['stage1', 'stage2', 'stage3', 'playoffs']

def list_model_directories():
    current_dir = os.getcwd()
    print(f"Current directory: {current_dir}")
    
    # List all items in the current directory
    model_dirs = [d for d in os.listdir() if os.path.isdir(d) and not d.startswith('stats')]
    print(f"Found model directories: {model_dirs}")
    return model_dirs

def get_team_stats(model: str, team: str, stage: str) -> Optional[dict]:
    try:
        file_path = f"{model}/{stage}/championship-cached/{team}.json"
        with open(file_path, 'r') as file:
            return json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        return None

def subtract_previous_results(current: dict, previous: dict) -> dict:
    if not previous:
        return current
    
    # Deep copy to avoid modifying original
    result = current.copy()
    
    # Subtract wins and losses
    result['wins'] = current['wins'] - previous['wins']
    result['losses'] = current['losses'] - previous['losses']
    
    # Subtract wins_against
    current_wins = set(x.strip() for x in current['win over'].split(',') if x.strip())
    previous_wins = set(x.strip() for x in previous['win over'].split(',') if x.strip())
    new_wins = current_wins - previous_wins
    result['win over'] = ', '.join(new_wins)
    
    # Subtract losses_against
    current_losses = set(x.strip() for x in current['loss over'].split(',') if x.strip())
    previous_losses = set(x.strip() for x in previous['loss over'].split(',') if x.strip())
    new_losses = current_losses - previous_losses
    result['loss over'] = ', '.join(new_losses)
    
    return result

def generate_markdown_report(stage_stats: Dict[str, dict], stage: str) -> str:
    markdown = f"### {stage}\n\n"
    
    if stage_stats:
        markdown += "| Team | W-L | New Wins Against | New Losses Against |\n"
        markdown += "|------|-----|-----------------|------------------|\n"
        
        # Sort teams by wins and losses
        sorted_teams = sorted(
            stage_stats.items(),
            key=lambda x: (x[1]['wins'], -x[1]['losses']),
            reverse=True
        )
        
        for team, stats in sorted_teams:
            markdown += f"| {team} | +{stats['wins']}-{stats['losses']} | "
            markdown += f"{stats['win over'] or '-'} | {stats['loss over'] or '-'} |\n"
    else:
        markdown += "*No new results for this stage*\n"
    
    markdown += "\n"
    return markdown

# Collect all data first
models_data = {}

for model in list_model_directories():
    models_data[model] = {}
    previous_stage_data = {}
    
    for stage in stages:
        stage_stats = {}
        
        for team in teams:
            current_data = get_team_stats(model, team, stage)
            if current_data:
                prev_data = previous_stage_data.get(team)
                
                if prev_data:
                    incremental_data = subtract_previous_results(current_data, prev_data)
                else:
                    incremental_data = current_data
                
                if incremental_data['wins'] > 0 or incremental_data['losses'] > 0:
                    stage_stats[team] = incremental_data
                
                previous_stage_data[team] = current_data
        
        if stage_stats:
            models_data[model][stage] = stage_stats

# Generate and save individual README files for each model
for model, stages_data in models_data.items():
    markdown_content = f"# {model} Results\n\n"
    
    for stage in stages:
        stage_stats = stages_data.get(stage, {})
        markdown_content += generate_markdown_report(stage_stats, stage)
    
    # Save to model-specific README
    readme_path = os.path.join(model, 'README.md')
    with open(readme_path, 'w') as f:
        f.write(markdown_content)
    
    print(f"Created README.md for {model}")

print("\nAll README files have been created")